# Title:  PRCP-1006-Home Loan Default - Risk Management

# Proble statement:  Contains multiple databases and sources to predict how capable each loan applicant is competent in repaying the loan.

# Description: The dataset contains nearly 671 MB input data in 7 different CSV files. Find relations between supplied inputs and join the meaningful inputs with appropriate grouping for modeling and prediction. 

### Importing essential libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_rows', 50000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.width', 50000)

### Loading application_train, bureau_balance, bureau & previous_application data frames for pre-processing 

In [2]:
at=pd.read_csv('application_train.csv')

In [3]:
bb=pd.read_csv('bureau_balance.csv')

In [4]:
bu=pd.read_csv('bureau.csv')

In [5]:
pa=pd.read_csv('previous_application.csv')

### Input size and property check

In [6]:
at.shape

(307511, 122)

In [7]:
at.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


In [8]:
bb.shape

(27299925, 3)

In [9]:
bb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27299925 entries, 0 to 27299924
Data columns (total 3 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   SK_ID_BUREAU    int64 
 1   MONTHS_BALANCE  int64 
 2   STATUS          object
dtypes: int64(2), object(1)
memory usage: 624.8+ MB


In [10]:
bu.shape

(1716428, 17)

In [11]:
bu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716428 entries, 0 to 1716427
Data columns (total 17 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   SK_ID_CURR              int64  
 1   SK_ID_BUREAU            int64  
 2   CREDIT_ACTIVE           object 
 3   CREDIT_CURRENCY         object 
 4   DAYS_CREDIT             int64  
 5   CREDIT_DAY_OVERDUE      int64  
 6   DAYS_CREDIT_ENDDATE     float64
 7   DAYS_ENDDATE_FACT       float64
 8   AMT_CREDIT_MAX_OVERDUE  float64
 9   CNT_CREDIT_PROLONG      int64  
 10  AMT_CREDIT_SUM          float64
 11  AMT_CREDIT_SUM_DEBT     float64
 12  AMT_CREDIT_SUM_LIMIT    float64
 13  AMT_CREDIT_SUM_OVERDUE  float64
 14  CREDIT_TYPE             object 
 15  DAYS_CREDIT_UPDATE      int64  
 16  AMT_ANNUITY             float64
dtypes: float64(8), int64(6), object(3)
memory usage: 222.6+ MB


In [12]:
pa.shape

(1670214, 37)

In [13]:
bb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27299925 entries, 0 to 27299924
Data columns (total 3 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   SK_ID_BUREAU    int64 
 1   MONTHS_BALANCE  int64 
 2   STATUS          object
dtypes: int64(2), object(1)
memory usage: 624.8+ MB


In [14]:
pa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1670214 entries, 0 to 1670213
Data columns (total 37 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   SK_ID_PREV                   1670214 non-null  int64  
 1   SK_ID_CURR                   1670214 non-null  int64  
 2   NAME_CONTRACT_TYPE           1670214 non-null  object 
 3   AMT_ANNUITY                  1297979 non-null  float64
 4   AMT_APPLICATION              1670214 non-null  float64
 5   AMT_CREDIT                   1670213 non-null  float64
 6   AMT_DOWN_PAYMENT             774370 non-null   float64
 7   AMT_GOODS_PRICE              1284699 non-null  float64
 8   WEEKDAY_APPR_PROCESS_START   1670214 non-null  object 
 9   HOUR_APPR_PROCESS_START      1670214 non-null  int64  
 10  FLAG_LAST_APPL_PER_CONTRACT  1670214 non-null  object 
 11  NFLAG_LAST_APPL_IN_DAY       1670214 non-null  int64  
 12  RATE_DOWN_PAYMENT            774370 non-nu

In [15]:
print("Total records in bureau.csv",bu.shape,"\n")
print("unique bureau customer ID",bu.SK_ID_CURR.unique().shape,"\n")
print("unique bureau ID",bu.SK_ID_BUREAU.unique().shape,"\n")
print("==================================================================\n")
print("Total records in bureau_balance.csv",bb.shape,"\n")
print("unique bureau balance bureau ID",bb.SK_ID_BUREAU.unique().shape,"\n")
print("==================================================================\n")
print("Total records in previous_application.csv",pa.shape,"\n")
print("unique previous_application customer ID",pa.SK_ID_CURR.unique().shape,"\n")

Total records in bureau.csv (1716428, 17) 

unique bureau customer ID (305811,) 

unique bureau ID (1716428,) 


Total records in bureau_balance.csv (27299925, 3) 

unique bureau balance bureau ID (817395,) 


Total records in previous_application.csv (1670214, 37) 

unique previous_application customer ID (338857,) 



In [16]:
# compare SK_ID_Curr that are in Previous application with BUreau table SK_ID_Curr 
# to ensure all the previous application history is avaible in bureau also
pa_count = pa.groupby(['SK_ID_CURR']).size().to_frame('PA_CID_CNT')
bu_count = pa.groupby(['SK_ID_CURR']).size().to_frame('BU_CID_CNT')
pa_bu_CID_merge = pd.merge(pa_count,bu_count, left_index=True, right_index=True).reset_index()

pa_bu_CID_merge.head()

,SK_ID_CURR,PA_CID_CNT,BU_CID_CNT
0,100001,1,1
1,100002,1,1
2,100003,3,3
3,100004,1,1
4,100005,2,2


In [17]:
pa_bu_CID_merge.shape

(338857, 3)

In [18]:
pa_bu_CID_merge.isnull().sum()

SK_ID_CURR    0
PA_CID_CNT    0
BU_CID_CNT    0
dtype: int64

In [19]:
pa_bu_CID_merge[pa_bu_CID_merge["BU_CID_CNT"]==0]

,SK_ID_CURR,PA_CID_CNT,BU_CID_CNT


In [20]:
pa_bu_CID_merge[pa_bu_CID_merge["PA_CID_CNT"]==0]

,SK_ID_CURR,PA_CID_CNT,BU_CID_CNT


## Early assessment summary: No NaN values post merge bureau and previous application

## Based on business knowledge we already deducted the defaulters; our assumption is when all the previous customers ids present in bureau means we already have 100% history of the applicants details in these data sets (application_train, bureau_balance, bureau) hence ignoring  POS_CASH_balance, credit_card_balance, installments_payments.

# Phase 1: Data Processing Bureau Balance to identify defaulters

In [21]:
# Finding unique status values

bureaU_status = bb.STATUS.unique()
print (bureaU_status)

['C' '0' 'X' '1' '2' '3' '5' '4']


### Encoding bureau_balance.csv status column

In [22]:
def bb_label (row):
    if row['STATUS'] == 'C': return 0
    if row['STATUS'] == 'X': return 0
    if row['STATUS'] == '0': return 0
    return 1

In [23]:
bb['bb_label'] = bb.apply (lambda row: bb_label(row), axis=1)

In [24]:
bb.shape

(27299925, 4)

In [25]:
NbureaU_status = bb.bb_label.unique()
print (NbureaU_status)

[0 1]


In [26]:
# to group the bureau by bureau ID removing insignificant months balance column

bb.drop(['MONTHS_BALANCE'], axis=1,inplace=True)

In [27]:
# ensuring the shape
bb.shape

(27299925, 3)

### Grouping bureau encoded labels by sum and count index

In [30]:
a = bb.groupby(['SK_ID_BUREAU'])['bb_label'].sum()
b = bb.groupby(['SK_ID_BUREAU']).size().to_frame('count')

bb_n = pd.merge(a,b, left_index=True, right_index=True).reset_index()

In [29]:
# ensuring size after grouping

bb_n.shape

(817395, 3)

In [25]:
# Verifying sample bureau ids and its count for grouping
# 5001786, 5011459

s_bb = bb[bb["SK_ID_BUREAU"]==6842888]
s_bb

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS,bb_label
23544003,6842888,0,C,0
23544004,6842888,-1,C,0
23544005,6842888,-2,C,0
23544006,6842888,-3,C,0
23544007,6842888,-4,C,0
23544008,6842888,-5,C,0
23544009,6842888,-6,C,0
23544010,6842888,-7,C,0
23544011,6842888,-8,C,0
23544012,6842888,-9,C,0


### Merge b/w bureau_balance.csv and bureau.csv

In [31]:
# Join b/w bureau_balance.csv and bureau.csv
bu_n = pd.merge(bu,bb_n, left_index=True, right_index=True).reset_index()

In [32]:
bu_n.shape

(817395, 21)

In [33]:
bu_n.head()

,index,SK_ID_CURR,SK_ID_BUREAU_x,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,SK_ID_BUREAU_y,bb_label,count
0,0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,5001709,0,97
1,1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,5001710,0,83
2,2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,5001711,0,4
3,3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,5001712,0,19
4,4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,5001713,0,22


In [34]:
# Verifying 
bu_np = bu_n[bu_n["SK_ID_BUREAU_x"]==5714462]
bu_np.head()

,index,SK_ID_CURR,SK_ID_BUREAU_x,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,SK_ID_BUREAU_y,bb_label,count
0,0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,5001709,0,97


### Phase 1: Complete 

# Phase 2: Data Processing combined Bureau and Bureau Balance

In [35]:
# Identifying categorical cols for encoding

cat_features = bu_n.select_dtypes(include='object').columns.tolist()
cat_features

['CREDIT_ACTIVE', 'CREDIT_CURRENCY', 'CREDIT_TYPE']

In [36]:
# Create Subset categorical features from bureau data

bureau_cat = bu_n[cat_features] 
bureau_cat

,CREDIT_ACTIVE,CREDIT_CURRENCY,CREDIT_TYPE
0,Closed,currency 1,Consumer credit
1,Active,currency 1,Credit card
2,Active,currency 1,Consumer credit
3,Active,currency 1,Credit card
4,Active,currency 1,Consumer credit
...,...,...,...
817390,Active,currency 1,Mortgage
817391,Closed,currency 1,Credit card
817392,Closed,currency 1,Car loan
817393,Closed,currency 2,Mortgage


In [37]:
# applying onehot encoding to the categorical fetures subset
onehot_encoder = OneHotEncoder(sparse=False, handle_unknown = 'ignore')
bureau_cat_encode = pd.DataFrame(onehot_encoder.fit_transform(bureau_cat))
bureau_cat_encode

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817390,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
817391,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
817392,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
817393,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [38]:
feature_encode = onehot_encoder.get_feature_names(cat_features)
bureau_cat_encode.columns = feature_encode
bureau_cat_encode

,CREDIT_ACTIVE_Active,CREDIT_ACTIVE_Bad debt,CREDIT_ACTIVE_Closed,CREDIT_ACTIVE_Sold,CREDIT_CURRENCY_currency 1,CREDIT_CURRENCY_currency 2,CREDIT_CURRENCY_currency 3,CREDIT_CURRENCY_currency 4,CREDIT_TYPE_Another type of loan,CREDIT_TYPE_Car loan,CREDIT_TYPE_Cash loan (non-earmarked),CREDIT_TYPE_Consumer credit,CREDIT_TYPE_Credit card,CREDIT_TYPE_Interbank credit,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Loan for purchase of shares (margin lending),CREDIT_TYPE_Loan for the purchase of equipment,CREDIT_TYPE_Loan for working capital replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile operator loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817390,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
817391,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
817392,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
817393,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [39]:
# Extract numerical features and combine into the encoded categorical data

num_features = ['SK_ID_CURR', 'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE', 'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_OVERDUE', 'DAYS_CREDIT_UPDATE', 'DAYS_CREDIT']
bureau_num = bu[num_features] 
bureau_proc = pd.concat([bureau_num, bureau_cat_encode], axis=1)
print("==shape=check==")
print(bureau_proc.shape)
print(bureau_num.shape)
print(bureau_cat_encode.shape)

==shape=check==
(1716428, 32)
(1716428, 9)
(817395, 23)


In [40]:
# Group by SK_ID_CURR and get sum & mean of other bureau data columns

bureau_proc_sum = bureau_proc.groupby(['SK_ID_CURR']).sum()
bureau_proc_mean = bureau_proc.groupby(['SK_ID_CURR']).mean()

In [41]:
# Verification
bureau_proc_sum.head(15)

,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,DAYS_CREDIT,CREDIT_ACTIVE_Active,CREDIT_ACTIVE_Bad debt,CREDIT_ACTIVE_Closed,CREDIT_ACTIVE_Sold,CREDIT_CURRENCY_currency 1,CREDIT_CURRENCY_currency 2,CREDIT_CURRENCY_currency 3,CREDIT_CURRENCY_currency 4,CREDIT_TYPE_Another type of loan,CREDIT_TYPE_Car loan,CREDIT_TYPE_Cash loan (non-earmarked),CREDIT_TYPE_Consumer credit,CREDIT_TYPE_Credit card,CREDIT_TYPE_Interbank credit,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Loan for purchase of shares (margin lending),CREDIT_TYPE_Loan for the purchase of equipment,CREDIT_TYPE_Loan for working capital replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile operator loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,-5145,0,577.0,0,1453365.000,0.0,-652,-5145,3.0,0.0,4.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100002,-6992,0,-2094.0,0,865055.565,0.0,-3999,-6992,1.0,0.0,5.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100003,-5603,0,-2178.0,0,1017400.500,0.0,-3264,-5603,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100004,-1734,0,-977.0,0,189037.800,0.0,-1064,-1734,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100005,-572,0,1318.0,0,657126.000,0.0,-163,-572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100007,-1149,0,-783.0,0,146250.000,0.0,-783,-1149,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100008,-2272,0,-1174.0,0,468445.500,0.0,-1833,-2272,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100009,-22887,0,-12719.0,0,4800811.500,0.0,-15329,-22887,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100010,-3879,0,-239.0,0,990000.000,0.0,-1156,-3879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
print(bureau_proc_sum.shape)
print (bureau_proc_mean.shape)

(305811, 31)
(305811, 31)


In [43]:
# merging both means and sum values
bureau_proc_final = pd.merge(bureau_proc_sum, bureau_proc_mean, how='left', on='SK_ID_CURR')
bureau_proc_final.shape

(305811, 62)

In [44]:
# writing final encoded file as csv
bureau_proc_final.to_csv('bureau_proc_final.csv', index=True)

In [45]:
# reading the file to hold the memory

bu_final = pd.read_csv('bureau_proc_final.csv')

### Phase 2 Complete

# Phase 3: Data processing Application train

In [46]:
# Check if 'SK_ID_CURR' in the train data
'SK_ID_CURR' in at.columns

True

### Merge bureau_proc_final to application data

In [47]:
at_merged = pd.merge(at, bu_final, how='left', on='SK_ID_CURR')
print(at.shape)
print(at_merged.shape)

(307511, 122)
(307511, 184)


In [48]:
at_merged.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_CREDIT_x,CREDIT_DAY_OVERDUE_x,DAYS_CREDIT_ENDDATE_x,CNT_CREDIT_PROLONG_x,AMT_CREDIT_SUM_x,AMT_CREDIT_SUM_OVERDUE_x,DAYS_CREDIT_UPDATE_x,DAYS_CREDIT_x.1,CREDIT_ACTIVE_Active_x,CREDIT_ACTIVE_Bad debt_x,CREDIT_ACTIVE_Closed_x,CREDIT_ACTIVE_Sold_x,CREDIT_CURRENCY_currency 1_x,CREDIT_CURRENCY_currency 2_x,CREDIT_CURRENCY_currency 3_x,CREDIT_CURRENCY_currency 4_x,CREDIT_TYPE_Another type of loan_x,CREDIT_TYPE_Car loan_x,CREDIT_TYPE_Cash loan (non-earmarked)_x,CREDIT_TYPE_Consumer credit_x,CREDIT_TYPE_Credit card_x,CREDIT_TYPE_Interbank credit_x,CREDIT_TYPE_Loan for business development_x,CREDIT_TYPE_Loan for purchase of shares (margin lending)_x,CREDIT_TYPE_Loan for the purchase of equipment_x,CREDIT_TYPE_Loan for working capital replenishment_x,CREDIT_TYPE_Microloan_x,CREDIT_TYPE_Mobile operator loan_x,CREDIT_TYPE_Mortgage_x,CREDIT_TYPE_Real estate loan_x,CREDIT_TYPE_Unknown type of loan_x,DAYS_CREDIT_y,CREDIT_DAY_OVERDUE_y,DAYS_CREDIT_ENDDATE_y,CNT_CREDIT_PROLONG_y,AMT_CREDIT_SUM_y,AMT_CREDIT_SUM_OVERDUE_y,DAYS_CREDIT_UPDATE_y,DAYS_CREDIT_y.1,CREDIT_ACTIVE_Active_y,CREDIT_ACTIVE_Bad debt_y,CREDIT_ACTIVE_Closed_y,CREDIT_ACTIVE_Sold_y,CREDIT_CURRENCY_currency 1_y,CREDIT_CURRENCY_currency 2_y,CREDIT_CURRENCY_currency 3_y,CREDIT_CURRENCY_currency 4_y,CREDIT_TYPE_Another type of loan_y,CREDIT_TYPE_Car loan_y,CREDIT_TYPE_Cash loan (non-earmarked)_y,CREDIT_TYPE_Consumer credit_y,CREDIT_TYPE_Credit card_y,CREDIT_TYPE_Interbank credit_y,CREDIT_TYPE_Loan for business development_y,CREDIT_TYPE_Loan for purchase of shares (margin lending)_y,CREDIT_TYPE_Loan for the purchase of equipment_y,CREDIT_TYPE_Loan for working capital replenishment_y,CREDIT_TYPE_Microloan_y,CREDIT_TYPE_Mobile operator loan_y,CREDIT_TYPE_Mortgage_y,CREDIT_TYPE_Real estate loan_y,CREDIT_TYPE_Unknown type of

### Extract int, float and categorical features for further processing: impute, scaling, one-hot encoding

In [49]:
cat_features = at_merged.select_dtypes(include='object').columns.tolist()

In [50]:
cat_features

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE']

In [51]:
# checking cat_features inside
for x in cat_features:
    print(at_merged[x].value_counts())

Cash loans         278232
Revolving loans     29279
Name: NAME_CONTRACT_TYPE, dtype: int64
F      202448
M      105059
XNA         4
Name: CODE_GENDER, dtype: int64
N    202924
Y    104587
Name: FLAG_OWN_CAR, dtype: int64
Y    213312
N     94199
Name: FLAG_OWN_REALTY, dtype: int64
Unaccompanied      248526
Family              40149
Spouse, partner     11370
Children             3267
Other_B              1770
Other_A               866
Group of people       271
Name: NAME_TYPE_SUITE, dtype: int64
Working                 158774
Commercial associate     71617
Pensioner                55362
State servant            21703
Unemployed                  22
Student                     18
Businessman                 10
Maternity leave              5
Name: NAME_INCOME_TYPE, dtype: int64
Secondary / secondary special    218391
Higher education                  74863
Incomplete higher                 10277
Lower secondary                    3816
Academic degree                     164
Name: NAME_EDUC

In [52]:
from sklearn.preprocessing import LabelEncoder

In [53]:
col = ('Arch','Beam','Truss','Cantilever','Tied Arch','Suspension','Cable')
labelencoder = LabelEncoder()
newCol = labelencoder.fit_transform(col)
newCol

array([0, 1, 6, 3, 5, 4, 2], dtype=int64)

In [54]:
# fixing data type inconsitency issue due to pandas data type 'Object' indicates mixed types rather than str type
# TypeError: Encoders require their input to be uniformly strings or numbers. Got ['float', 'str']

In [55]:
le = LabelEncoder()
at_merged['NAME_CONTRACT_TYPE']= le.fit_transform(at_merged['NAME_CONTRACT_TYPE'])
at_merged['CODE_GENDER']= le.fit_transform(at_merged['CODE_GENDER'])
at_merged['FLAG_OWN_CAR']= le.fit_transform(at_merged['FLAG_OWN_CAR'])
at_merged['FLAG_OWN_REALTY']= le.fit_transform(at_merged['FLAG_OWN_REALTY'])
at_merged['NAME_TYPE_SUITE']= le.fit_transform(at_merged['NAME_TYPE_SUITE'].astype(str))
at_merged['NAME_INCOME_TYPE']= le.fit_transform(at_merged['NAME_INCOME_TYPE'])
at_merged['NAME_EDUCATION_TYPE']= le.fit_transform(at_merged['NAME_EDUCATION_TYPE'])
at_merged['NAME_FAMILY_STATUS']= le.fit_transform(at_merged['NAME_FAMILY_STATUS'])
at_merged['NAME_HOUSING_TYPE']= le.fit_transform(at_merged['NAME_HOUSING_TYPE'])
at_merged['OCCUPATION_TYPE']= le.fit_transform(at_merged['OCCUPATION_TYPE'].astype(str))
at_merged['WEEKDAY_APPR_PROCESS_START']= le.fit_transform(at_merged['WEEKDAY_APPR_PROCESS_START'])
at_merged['ORGANIZATION_TYPE']= le.fit_transform(at_merged['ORGANIZATION_TYPE'])
at_merged['FONDKAPREMONT_MODE']= le.fit_transform(at_merged['FONDKAPREMONT_MODE'].astype(str))
at_merged['HOUSETYPE_MODE']= le.fit_transform(at_merged['HOUSETYPE_MODE'].astype(str))
at_merged['WALLSMATERIAL_MODE']= le.fit_transform(at_merged['WALLSMATERIAL_MODE'].astype(str))
at_merged['EMERGENCYSTATE_MODE']= le.fit_transform(at_merged['EMERGENCYSTATE_MODE'].astype(str))

at_merged = pd.get_dummies(at_merged, dummy_na = True)

In [56]:
# checking insights of cat_features
for x in cat_features:
    print(at_merged[x].value_counts())

0    278232
1     29279
Name: NAME_CONTRACT_TYPE, dtype: int64
0    202448
1    105059
2         4
Name: CODE_GENDER, dtype: int64
0    202924
1    104587
Name: FLAG_OWN_CAR, dtype: int64
1    213312
0     94199
Name: FLAG_OWN_REALTY, dtype: int64
6    248526
1     40149
5     11370
0      3267
4      1770
7      1292
3       866
2       271
Name: NAME_TYPE_SUITE, dtype: int64
7    158774
1     71617
3     55362
4     21703
6        22
5        18
0        10
2         5
Name: NAME_INCOME_TYPE, dtype: int64
4    218391
1     74863
2     10277
3      3816
0       164
Name: NAME_EDUCATION_TYPE, dtype: int64
1    196432
3     45444
0     29775
2     19770
5     16088
4         2
Name: NAME_FAMILY_STATUS, dtype: int64
1    272868
5     14840
2     11183
4      4881
3      2617
0      1122
Name: NAME_HOUSING_TYPE, dtype: int64
18    96391
8     55186
14    32102
3     27570
10    21371
4     18603
6     11380
0      9813
11     8537
16     6721
2      5946
1      4653
12     2652
9      209

In [57]:
at_merged.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_CREDIT_x,CREDIT_DAY_OVERDUE_x,DAYS_CREDIT_ENDDATE_x,CNT_CREDIT_PROLONG_x,AMT_CREDIT_SUM_x,AMT_CREDIT_SUM_OVERDUE_x,DAYS_CREDIT_UPDATE_x,DAYS_CREDIT_x.1,CREDIT_ACTIVE_Active_x,CREDIT_ACTIVE_Bad debt_x,CREDIT_ACTIVE_Closed_x,CREDIT_ACTIVE_Sold_x,CREDIT_CURRENCY_currency 1_x,CREDIT_CURRENCY_currency 2_x,CREDIT_CURRENCY_currency 3_x,CREDIT_CURRENCY_currency 4_x,CREDIT_TYPE_Another type of loan_x,CREDIT_TYPE_Car loan_x,CREDIT_TYPE_Cash loan (non-earmarked)_x,CREDIT_TYPE_Consumer credit_x,CREDIT_TYPE_Credit card_x,CREDIT_TYPE_Interbank credit_x,CREDIT_TYPE_Loan for business development_x,CREDIT_TYPE_Loan for purchase of shares (margin lending)_x,CREDIT_TYPE_Loan for the purchase of equipment_x,CREDIT_TYPE_Loan for working capital replenishment_x,CREDIT_TYPE_Microloan_x,CREDIT_TYPE_Mobile operator loan_x,CREDIT_TYPE_Mortgage_x,CREDIT_TYPE_Real estate loan_x,CREDIT_TYPE_Unknown type of loan_x,DAYS_CREDIT_y,CREDIT_DAY_OVERDUE_y,DAYS_CREDIT_ENDDATE_y,CNT_CREDIT_PROLONG_y,AMT_CREDIT_SUM_y,AMT_CREDIT_SUM_OVERDUE_y,DAYS_CREDIT_UPDATE_y,DAYS_CREDIT_y.1,CREDIT_ACTIVE_Active_y,CREDIT_ACTIVE_Bad debt_y,CREDIT_ACTIVE_Closed_y,CREDIT_ACTIVE_Sold_y,CREDIT_CURRENCY_currency 1_y,CREDIT_CURRENCY_currency 2_y,CREDIT_CURRENCY_currency 3_y,CREDIT_CURRENCY_currency 4_y,CREDIT_TYPE_Another type of loan_y,CREDIT_TYPE_Car loan_y,CREDIT_TYPE_Cash loan (non-earmarked)_y,CREDIT_TYPE_Consumer credit_y,CREDIT_TYPE_Credit card_y,CREDIT_TYPE_Interbank credit_y,CREDIT_TYPE_Loan for business development_y,CREDIT_TYPE_Loan for purchase of shares (margin lending)_y,CREDIT_TYPE_Loan for the purchase of equipment_y,CREDIT_TYPE_Loan for working capital replenishment_y,CREDIT_TYPE_Microloan_y,CREDIT_TYPE_Mobile operator loan_y,CREDIT_TYPE_Mortgage_y,CREDIT_TYPE_Real estate loan_y,CREDIT_TYPE_Unknown type of

In [58]:
# X and y split 
X=at_merged.loc[:,['SK_ID_CURR','CNT_CHILDREN','DAYS_BIRTH','DAYS_EMPLOYED','DAYS_ID_PUBLISH','FLAG_MOBIL','FLAG_EMP_PHONE','FLAG_WORK_PHONE','FLAG_CONT_MOBILE','FLAG_PHONE','FLAG_EMAIL','REGION_RATING_CLIENT','REGION_RATING_CLIENT_W_CITY','HOUR_APPR_PROCESS_START','REG_REGION_NOT_LIVE_REGION','REG_REGION_NOT_WORK_REGION','LIVE_REGION_NOT_WORK_REGION','REG_CITY_NOT_LIVE_CITY','REG_CITY_NOT_WORK_CITY','LIVE_CITY_NOT_WORK_CITY','FLAG_DOCUMENT_2','FLAG_DOCUMENT_3','FLAG_DOCUMENT_4','FLAG_DOCUMENT_5','FLAG_DOCUMENT_6','FLAG_DOCUMENT_7','FLAG_DOCUMENT_8','FLAG_DOCUMENT_9','FLAG_DOCUMENT_10','FLAG_DOCUMENT_11','FLAG_DOCUMENT_12','FLAG_DOCUMENT_13','FLAG_DOCUMENT_14','FLAG_DOCUMENT_15','FLAG_DOCUMENT_16','FLAG_DOCUMENT_17','FLAG_DOCUMENT_18','FLAG_DOCUMENT_19','FLAG_DOCUMENT_20','FLAG_DOCUMENT_21','AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY','AMT_GOODS_PRICE','REGION_POPULATION_RELATIVE','DAYS_REGISTRATION','OWN_CAR_AGE','CNT_FAM_MEMBERS','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','APARTMENTS_AVG','BASEMENTAREA_AVG','YEARS_BEGINEXPLUATATION_AVG','YEARS_BUILD_AVG','COMMONAREA_AVG','ELEVATORS_AVG','ENTRANCES_AVG','FLOORSMAX_AVG','FLOORSMIN_AVG','LANDAREA_AVG','LIVINGAPARTMENTS_AVG','LIVINGAREA_AVG','NONLIVINGAPARTMENTS_AVG','NONLIVINGAREA_AVG','APARTMENTS_MODE','BASEMENTAREA_MODE','YEARS_BEGINEXPLUATATION_MODE','YEARS_BUILD_MODE','COMMONAREA_MODE','ELEVATORS_MODE','ENTRANCES_MODE','FLOORSMAX_MODE','FLOORSMIN_MODE','LANDAREA_MODE','LIVINGAPARTMENTS_MODE','LIVINGAREA_MODE','NONLIVINGAPARTMENTS_MODE','NONLIVINGAREA_MODE','APARTMENTS_MEDI','BASEMENTAREA_MEDI','YEARS_BEGINEXPLUATATION_MEDI','YEARS_BUILD_MEDI','COMMONAREA_MEDI','ELEVATORS_MEDI','ENTRANCES_MEDI','FLOORSMAX_MEDI','FLOORSMIN_MEDI','LANDAREA_MEDI','LIVINGAPARTMENTS_MEDI','LIVINGAREA_MEDI','NONLIVINGAPARTMENTS_MEDI','NONLIVINGAREA_MEDI','TOTALAREA_MODE','OBS_30_CNT_SOCIAL_CIRCLE','DEF_30_CNT_SOCIAL_CIRCLE','OBS_60_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE','DAYS_LAST_PHONE_CHANGE','AMT_REQ_CREDIT_BUREAU_HOUR','AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_WEEK','AMT_REQ_CREDIT_BUREAU_MON','AMT_REQ_CREDIT_BUREAU_QRT','AMT_REQ_CREDIT_BUREAU_YEAR','DAYS_CREDIT_x','CREDIT_DAY_OVERDUE_x','DAYS_CREDIT_ENDDATE_x','CNT_CREDIT_PROLONG_x','AMT_CREDIT_SUM_x','AMT_CREDIT_SUM_OVERDUE_x','DAYS_CREDIT_UPDATE_x','DAYS_CREDIT_x.1','CREDIT_ACTIVE_Active_x','CREDIT_ACTIVE_Bad debt_x','CREDIT_ACTIVE_Closed_x','CREDIT_ACTIVE_Sold_x','CREDIT_CURRENCY_currency 1_x','CREDIT_CURRENCY_currency 2_x','CREDIT_CURRENCY_currency 3_x','CREDIT_CURRENCY_currency 4_x','CREDIT_TYPE_Another type of loan_x','CREDIT_TYPE_Car loan_x','CREDIT_TYPE_Cash loan (non-earmarked)_x','CREDIT_TYPE_Consumer credit_x','CREDIT_TYPE_Credit card_x','CREDIT_TYPE_Interbank credit_x','CREDIT_TYPE_Loan for business development_x','CREDIT_TYPE_Loan for purchase of shares (margin lending)_x','CREDIT_TYPE_Loan for the purchase of equipment_x','CREDIT_TYPE_Loan for working capital replenishment_x','CREDIT_TYPE_Microloan_x','CREDIT_TYPE_Mobile operator loan_x','CREDIT_TYPE_Mortgage_x','CREDIT_TYPE_Real estate loan_x','CREDIT_TYPE_Unknown type of loan_x','DAYS_CREDIT_y','CREDIT_DAY_OVERDUE_y','DAYS_CREDIT_ENDDATE_y','CNT_CREDIT_PROLONG_y','AMT_CREDIT_SUM_y','AMT_CREDIT_SUM_OVERDUE_y','DAYS_CREDIT_UPDATE_y','DAYS_CREDIT_y.1','CREDIT_ACTIVE_Active_y','CREDIT_ACTIVE_Bad debt_y','CREDIT_ACTIVE_Closed_y','CREDIT_ACTIVE_Sold_y','CREDIT_CURRENCY_currency 1_y','CREDIT_CURRENCY_currency 2_y','CREDIT_CURRENCY_currency 3_y','CREDIT_CURRENCY_currency 4_y','CREDIT_TYPE_Another type of loan_y','CREDIT_TYPE_Car loan_y','CREDIT_TYPE_Cash loan (non-earmarked)_y','CREDIT_TYPE_Consumer credit_y','CREDIT_TYPE_Credit card_y','CREDIT_TYPE_Interbank credit_y','CREDIT_TYPE_Loan for business development_y','CREDIT_TYPE_Loan for purchase of shares (margin lending)_y','CREDIT_TYPE_Loan for the purchase of equipment_y','CREDIT_TYPE_Loan for working capital replenishment_y','CREDIT_TYPE_Microloan_y','CREDIT_TYPE_Mobile operator loan_y','CREDIT_TYPE_Mortgage_y','CREDIT_TYPE_Real estate loan_y','CREDIT_TYPE_Unknown type of loan_y','NAME_CONTRACT_TYPE','CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','NAME_TYPE_SUITE','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','OCCUPATION_TYPE','WEEKDAY_APPR_PROCESS_START','ORGANIZATION_TYPE','FONDKAPREMONT_MODE','HOUSETYPE_MODE','WALLSMATERIAL_MODE','EMERGENCYSTATE_MODE']]
y=at_merged.TARGET

In [59]:
X.head(5)

,SK_ID_CURR,CNT_CHILDREN,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_REGISTRATION,OWN_CAR_AGE,CNT_FAM_MEMBERS,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_CREDIT_x,CREDIT_DAY_OVERDUE_x,DAYS_CREDIT_ENDDATE_x,CNT_CREDIT_PROLONG_x,AMT_CREDIT_SUM_x,AMT_CREDIT_SUM_OVERDUE_x,DAYS_CREDIT_UPDATE_x,DAYS_CREDIT_x.1,CREDIT_ACTIVE_Active_x,CREDIT_ACTIVE_Bad debt_x,CREDIT_ACTIVE_Closed_x,CREDIT_ACTIVE_Sold_x,CREDIT_CURRENCY_currency 1_x,CREDIT_CURRENCY_currency 2_x,CREDIT_CURRENCY_currency 3_x,CREDIT_CURRENCY_currency 4_x,CREDIT_TYPE_Another type of loan_x,CREDIT_TYPE_Car loan_x,CREDIT_TYPE_Cash loan (non-earmarked)_x,CREDIT_TYPE_Consumer credit_x,CREDIT_TYPE_Credit card_x,CREDIT_TYPE_Interbank credit_x,CREDIT_TYPE_Loan for business development_x,CREDIT_TYPE_Loan for purchase of shares (margin lending)_x,CREDIT_TYPE_Loan for the purchase of equipment_x,CREDIT_TYPE_Loan for working capital replenishment_x,CREDIT_TYPE_Microloan_x,CREDIT_TYPE_Mobile operator loan_x,CREDIT_TYPE_Mortgage_x,CREDIT_TYPE_Real estate loan_x,CREDIT_TYPE_Unknown type of loan_x,DAYS_CREDIT_y,CREDIT_DAY_OVERDUE_y,DAYS_CREDIT_ENDDATE_y,CNT_CREDIT_PROLONG_y,AMT_CREDIT_SUM_y,AMT_CREDIT_SUM_OVERDUE_y,DAYS_CREDIT_UPDATE_y,DAYS_CREDIT_y.1,CREDIT_ACTIVE_Active_y,CREDIT_ACTIVE_Bad debt_y,CREDIT_ACTIVE_Closed_y,CREDIT_ACTIVE_Sold_y,CREDIT_CURRENCY_currency 1_y,CREDIT_CURRENCY_currency 2_y,CREDIT_CURRENCY_currency 3_y,CREDIT_CURRENCY_currency 4_y,CREDIT_TYPE_Another type of loan_y,CREDIT_TYPE_Car loan_y,CREDIT_TYPE_Cash loan (non-earmarked)_y,CREDIT_TYPE_Consumer credit_y,CREDIT_TYPE_Credit card_y,CREDIT_TYPE_Interbank credit_y,CREDIT_TYPE_Loan for business development_y,CREDIT_TYPE_Loan for purchase of shares (margin lending)_y,CREDIT_TYPE_Loan for the purchase of equipment_y,CREDIT_TYPE_Loan for working capital replenishment_y,CREDIT_TYPE_Microloan_y,CREDIT_TYPE_Mobile operator loan_y,CREDIT_TYPE_Mortgage_y,CREDIT_TYPE_Real estate loan_y,CREDIT_TYPE_Unknown type of loan_y,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE

In [60]:
y.head(5)

0    1
1    0
2    0
3    0
4    0
Name: TARGET, dtype: int64

In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split (X,y,test_size=0.20,random_state=55)

In [62]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(246008, 183)
(246008,)
(61503, 183)
(61503,)


In [63]:
# pip install xgboost

In [64]:
import xgboost as xgb
from xgboost import XGBClassifier

model = XGBClassifier(
    objective="binary:logistic",
    eval_metric="auc",
    learning_rate = 0.1,
    max_depth=14,
    gamma=0.1,
    min_child_weight=6,
    reg_lambda=2
)

model.fit(X_train, y_train)

C:\Users\Thiri\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=0.1, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=14, min_child_weight=6, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=2,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [66]:
y_predict = model.predict(X_test)

In [67]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_predict)

0.9180690372827341

In [68]:
print(roc_auc_score(y_train, model.predict_proba(X_train)[:,1]))

0.9851150946735607


Iteration1: XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=0.382, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.277, max_delta_step=0,
              max_depth=4, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, random_state=42, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)
              
              ROC_AUC_SCORE 0.7980395871480825
              
Iteration2: XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=0.25, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.277, max_delta_step=0,
              max_depth=5, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, random_state=42, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)
              
              ROC_AUC_SCORE  0.8226752449949796
              
Iteration3:   XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=0.25, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.15, max_delta_step=0,
              max_depth=5, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, random_state=42, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)
              
              ROC_AUC_SCORE 0.8015690228199415
              
Iteration3:   XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=0.25, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.299, max_delta_step=0,
              max_depth=5, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, random_state=42, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)
              
              ROC_AUC_SCORE 0.8250096369926772
              
 Iteration4:  XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=0.1, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.295, max_delta_step=0,
              max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, random_state=45, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)
              
              ROC_AUC_SCORE 0.860861913653125
              
Iteration12:  XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=0.1, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=12, min_child_weight=5, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)
              
              ROC_AUC_SCORE  0.9728282437780764
              
Iteration14:  XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=0.1, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=14, min_child_weight=6, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)
              
              ROC_AUC_SCORE  0.9902416796382311

In [70]:
# checking the model overfitting?

y_predict_train = model.predict(X_train)
accuracy_score(y_train, y_predict_train)

0.9429002308868004

## Conclusion: The chosen XGBClassifier model provides better accuracy for such classification problems and we successfully modelled the given problem with multiple data sets. 
## Through careful hyperparameter tuning of the XGBClassifier model we achieved 98.5% of RUC AUC accuracy for Train data and  91.8% accuracy on Test data. 